# c05-plates

*Purpose*: Variability in engineering systems exposes us to risk. To rigorously design for variability, we need to analyze data, model the variability, and study its effects on system performance. To practice this entire process, you will study the structural safety of a plate subject to buckling loads.


## Informed Consent

As a reminder, this course is part of a study of engineers' behavior. While not all parts of the course include data collection, we will analyze your responses to this homework as part of the research.

We will analyze your answers to this homework, and may quote this work as part of published research.

You can ask to have your responses excluded from the study after the interview by sending us an email. Before starting this assignment, do you consent to sharing your work with the study?


I agree to share my responses with the study

- (Please type your name here)


In [ ]:
import grama as gr
import pandas as pd
DF = gr.Intention()
%matplotlib inline

# For assertion
from pandas.api.types import is_numeric_dtype


## Background

This challenge continues the `c03-stang` challenge; we will start from the same dataset but will apply ideas of modeling and design under uncertainty.


# Assess Statistical Control

Before we studied the Stang et al. dataset using generic EDA techniques; let's revisit this dataset armed with the knowledge of statistical process monitoring.


In [ ]:
from grama.data import df_stang


### __q1__ Assess statistical control of `E` across thicknesses

Construct a control chart with groupings according to plate thickness. Assess the state of statistical control of the elasticity. Answer the questions under *observations* below.


In [ ]:
## TODO: Construct a control chart
(
    df_stang

)

*Observations*

- Is the variability of `E` under statistical control across plate thicknesses? How do you know?
  - (Your response here)
- Is the mean of `E` under statistical control across plate thicknesses? How do you know?
  - (Your response here)


## Follow-up experiment

We've seen that the Stang et al. data are certainly *not* under statistical control (and we have some hints as to what went wrong!). Let's imagine that manufacturing of these plates has ramped up, and we have access to a much larger dataset from this production line.

*Note*: The following data were simulated; they do not correspond to physical experiments.


In [ ]:
df_data = pd.read_csv("./data/c05-data.csv")
df_data

### Data Dictionary

Here is a data dictionary for the new `df_data`.

| Symbol | Variable | Meaning |
|---|---|---|
| `E` | Elasticity (ksi) | Mechanical property |
| `mu` | Poisson's ratio (-) | Mechanical property |
| `t` | Thickness (in) | Geometric property |
| `id_machine` | Machine identifier | Manufacturing variable |
| `id_specimen` | Specimen identifier | Manufacturing variable |
| `id_measurement` | Measurement (operator) identifier | Manufacturing variable |


### __q2__ Explore the experimental design

Answer the following questions to better understand the experimental design. Note that the same questions are posed within each cell and under *observations* below.

*Hint*: The verbs `tf_count()` and `tf_distinct()` will be very useful for answering some of these questions!


In [ ]:
## Q: What thicknesses were tested?


In [ ]:
## Q: How many unique specimens were manufactured?


In [ ]:
## Q: How many specimens were made on each machine?


In [ ]:
## Q: How many times did each operator measure each specimen?


*Observations*

- What thicknesses were tested?
  - (Your response here)
- How many unique specimens were manufactured?
  - (Your response here)
- How many specimens were made on each machine?
  - (Your response here)
- How many times did each operator measure each specimen?
  - (Your response here)


### __q3__ Compare across thicknesses

Compare the elasticity across plate thicknesses; does elasticity seem to be consistent across thickness?


In [ ]:
## TODO: Compare elasticity across thicknesses
## NOTE: There are many ways to do this!
(
    df_data

)

*Observations*

- Is thickness consistent across plate thickness?
  - (Your response here)
- Will it be reasonable to group together plates of different thicknesses when assessing statistical control? Why or why not?
  - (Your response here)


### __q4__ Assess statistical control of Poisson's ratio

Consider only the `t == 0.250` plates. Assess the state of statistical control of Poisson's ratio. Answer the questions under *observations* below.


In [ ]:
## TODO: Assess the state of statistical control


*Observations*

- Is `mu` under statistical control? Why or why not?
  - (Your response here)


### __q5__ Assess statistical control of elasticity

Consider only the `t == 0.250` plates. Assess the state of statistical control of the elasticity. Answer the questions under *observations* below.


In [ ]:
## TODO: Assess the state of statistical control


*Observations*

- Why is it important that we limit this analysis to `t == 0.25` plates?
  - (Your response here)
- Is `E` under statistical control? Why or why not?
  - (Your response here)
- Based on the group variable(s) you chose, what follow-up investigations should be done?
  - (Your response here)


### __q6__ Assess statistical control of elasticity (Pt. 2)

Repeat your investigation of statistical control of `E`, but use the filters given below. Answer the questions under *observations* below.


In [ ]:
## TASK: Assess statistical control of `E` for the filtered data
(
    df_data
    >> gr.tf_filter(
        DF.t == 0.250,
        DF.id_machine != "C",
        DF.id_measurement != "h",
    )

)

In [ ]:
## TASK: Assess statistical control of `E` for the filtered data
(
    df_data
    >> gr.tf_filter(
        DF.t == 0.250,
        DF.id_machine != "C",
        DF.id_measurement != "h",
    )

)

*Observations*

- For the set of machines and operators considered, is `E` under statistical control? Why or why not?
  - (Your response here)


# Consider Sources of Variability

Now that we've found a more stable subset of the data, we can move towards modeling the variability with distributions. However, before we go much further, we should investigate the *sources* of variability in the data. Note that we have access to multiple measurements of the same specimens (identified by `id_measurement`); thus we can use the dataset to approximate the real and erroneous variability in the material properties.

For the rest of the exercise, we will consider the following subset of the data.


In [ ]:
## NOTE: No need to edit
df_sub = (
    df_data
    >> gr.tf_filter(
        DF.t == 0.250,
        DF.id_machine != "C",
        DF.id_measurement != "h",
    )
)

### __q7__ Estimate the real variability

Identify the column in `df_sub` that groups together multiple measurements of the same quantity.

The code below applies the *mean heuristic* ([e-stat02-source](https://zdelrosario.github.io/evc-course/exercises_solution/d08-e-stat02-source-solution.html#heuristics)) to produce a more stable measurement of `E`, then computes the variance across these more stable measurements.


In [ ]:
## TASK: Apply the Mean Heuristic to group by the appropriate variable
df_var_mfg = (
    df_sub
    ## TODO: Group by the appropriate variable

    >> gr.tf_summarize(E=gr.mean(DF.E))
    >> gr.tf_ungroup()
    >> gr.tf_summarize(E_var_mfg=gr.var(DF.E))
)

## NOTE: No need to edit; use this to check your work
assert \
    abs(df_var_mfg.E_var_mfg[0] - 109670.635716) < 1e-6, \
    "Incorrect variance; make sure you grouped by the correct variable"

df_var_mfg

### __q8__ Estimate the measurement variability

Estimate the measurement variability in `E` by taking the variance within each specimen. Average these variances to produce a more stable estimate `E_var_meas`. Answer the questions under *observations* below.


In [ ]:
## TASK: Estimate the measurement variability
df_var_meas = (
    df_sub

)

## NOTE: No need to edit; use this to check your work
assert \
    abs(df_var_meas.iloc[0, 0] - 168382.91387) < 1e-6, \
    "Incorrect variance; make sure you grouped by the correct variable"

df_var_meas

*Observations*

- How do `E_var_mfg` (previous task) and `E_var_meas` compare?
  - (Your response here)


The following code applies the mean heuristic to get more stable measurements for `E` and `mu`.


In [ ]:
## NOTE: No need to edit
df_real = (
    df_sub
    >> gr.tf_group_by(DF.id_specimen)
    >> gr.tf_summarize(
        mu=gr.mean(DF.mu),
        E=gr.mean(DF.E),
    )
    >> gr.tf_ungroup()
)

# Model the Variability

Now you will construct a model for the variability in `E` and `mu`. You'll use this to assess the structural safety of a plate subject to compressive (buckling) loads.


### __q9__ Assess dependency of `E` and `mu`

Assess the dependency between `E` and `mu`. Answer the questions under *observations* below.

*Hint*: There are many ways to do this!


In [ ]:
## TASK: Assess the dependency between E and mu
(
    df_sub

)

*Observations*

- What---if any---dependency do `E` and `mu` exhibit?
  - (Your response here)


The following code implements the buckling plate model; you wrote code like this in c03-stang.


In [ ]:
## NOTE: No need to edit
md_plate = (
    gr.Model("Plate critical buckling stress")
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            k_cr=(df.m * df.b / df.a + df.a / df.m / df.b)**2
        ),
        var=["a", "b", "m"],
        out=["k_cr"],
        name="Shape factor",
    )
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            sigma_cr=df.k_cr * (3.14**3/12) * df.E*1e3 / (1 - df.mu**2)
                    *(df.t / df.b)**2
        ),
        var=["k_cr", "E", "mu", "t", "b"],
        out=["sigma_cr"],
        name="Buckling stress",
    )
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            g_buckle=df.sigma_cr - 2e5 / df.b / df.t,
        ),
        var=["sigma_cr", "b", "t"],
        out=["g_buckle"],
        name="Limit state: Buckling",
    )
)

md_plate

### __q10__ Fit a distribution with all observations

Fit a distribution for the inputs `E` and `mu` to `md_plate` using the filtered dataset `df_sub`. Make sure to follow the proper modeling process. Add as many code cells as you need for this task. Answer the questions under *Observations* below.


In [ ]:
## TASK: Add your distribution model to `md_total`
md_total = (
    md_plate

)
md_total

*Observations*

- What assumptions / choices did you make in your model?
  - (Your response here)


### __q11__ Fit a distribution with averaged observations

Re-fit the same model you defined above, but use the averaged observations `df_real` instead.


In [ ]:
## TASK: Add your distribution model to `md_real`, using `df_real` in the fit
md_real = (
    md_plate

)
md_real

# Design Under Uncertainty

Now that we've built a couple models, we can use them to assess the structural safety of plate designs. We'll start by assessing the safety of a baseline design.


In [ ]:
## NOTE: No need to edit this
df_baseline = gr.df_make(t=0.25, a=12.0, b=9.0, m=1)


### __q12__ Assess a baseline design

Assess the probability of failure $\text{pof} = \mathbb{P}[g \leq 0]$ according to both `md_total` and `md_baseline`. Answer the questions under *observations* below.


In [ ]:
## TASK: Assess the probability of failure
df_baseline_total = (
    md_total

)

## NOTE: Use this to check your work
assert \
    isinstance(df_baseline_total, pd.DataFrame), \
    "df_baseline_total is not a DataFrame; make sure to evaluate the model"
assert \
    "pof_lo" in df_baseline_total.columns, \
    "df_baseline_total does not have a pof_lo column; make sure to include a lower CI end"
assert \
    "pof_up" in df_baseline_total.columns, \
    "df_baseline_total does not have a pof_up column; make sure to include a lower CI end"


In [ ]:
## TASK: Assess the probability of failure
df_baseline_real = (
    md_real

)

## NOTE: Use this to check your work
assert \
    isinstance(df_baseline_real, pd.DataFrame), \
    "df_baseline_real is not a DataFrame; make sure to evaluate the model"
assert \
    "pof_lo" in df_baseline_real.columns, \
    "df_baseline_real does not have a pof_lo column; make sure to include a lower CI end"
assert \
    "pof_up" in df_baseline_real.columns, \
    "df_baseline_real does not have a pof_up column; make sure to include a lower CI end"


In [ ]:
## NOTE: No need to edit; use this to check your work
(
    df_baseline_total
    >> gr.tf_mutate(model="Total")
    >> gr.tf_bind_rows(
        df_baseline_real
        >> gr.tf_mutate(model="Real")
    )
    
    >> gr.ggplot(gr.aes("model", "pof"))
    + gr.geom_hline(yintercept=0.01, linetype="dashed")
    + gr.geom_errorbar(gr.aes(ymin="pof_lo", ymax="pof_up"))
    + gr.geom_point()
)

*Observations*

- According to the `Total` model, does the baseline design meet the desired criteria of `pof < 0.01` (dashed line)?
  - (Your response here)
- According to the `Real` model, does the baseline design meet the desired criteria of `pof < 0.01` (dashed line)?
  - (Your response here)


### __q13__ Adjust the design

Adjust the thickness of the plate to *confidently* achieve `pof < 0.01`. Repeat this process for both the `Total` and `Real` models. Answer the questions under *observations* below.


In [ ]:
## TASK: Adjust the design
df_revised_total = gr.df_make(
    ## TODO: Adjust the thickness to modify the design
    # t=0.25,

    ## NOTE: Do not edit the following values
    a=12.0, b=9.0, m=1
)

## NOTE: No need to edit
df_revised_total = (
    md_total
    >> gr.ev_sample(n=1e4, df_det=df_revised_total)
    >> gr.tf_summarize(
        pof_lo=gr.pr_lo(DF.g_buckle <= 0),
        pof=gr.pr(DF.g_buckle <= 0),
        pof_up=gr.pr_up(DF.g_buckle <= 0),
    )
)

df_revised_total


In [ ]:
## TASK: Adjust the design
df_revised_real = gr.df_make(
    ## TODO: Adjust the thickness to modify the design
    # t=0.25,

    ## NOTE: Do not edit the following values
    a=12.0, b=9.0, m=1
)

## NOTE: No need to edit
df_revised_real = (
    md_real
    >> gr.ev_sample(n=1e4, df_det=df_revised_real)
    >> gr.tf_summarize(
        pof_lo=gr.pr_lo(DF.g_buckle <= 0),
        pof=gr.pr(DF.g_buckle <= 0),
        pof_up=gr.pr_up(DF.g_buckle <= 0),
    )
)

df_revised_real


In [ ]:
## NOTE: No need to edit; the following visual will help you assess the results
(
    df_revised_total
    >> gr.tf_mutate(model="Total")
    >> gr.tf_bind_rows(
        df_revised_real
        >> gr.tf_mutate(model="Real")
    )
    
    >> gr.ggplot(gr.aes("model", "pof"))
    + gr.geom_hline(yintercept=0.01, linetype="dashed")
    + gr.geom_errorbar(gr.aes(ymin="pof_lo", ymax="pof_up"))
    + gr.geom_point()
)

*Observations*

- What needs to be the case with your results to *confidently* conclude that `pof < 0.01`?
  - (Your response here)
- What thickness is necessary to confidently achieve `pof < 0.01` with the `Total` model?
  - (Your response here)
- What thickness is necessary to confidently achieve `pof < 0.01` with the `Real` model?
  - (Your response here)
- Suppose the plate manufacturing process can only achieve tolerances to within `0.01` in of thickness. Does distinguishing between real and total variability have a large consequence in this case?
  - (Your response here)
- Now suppose the plate manufacturing process can achieve tolerances to within `0.001` in of thickness, and weight is a major concern. Does distinguishing between real and total variability have a large consequence in this case?
  - (Your response here)
